In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output

color_map = {'B': 'Blue', 'Y': 'Yellow', 'O': 'Orange', 'R': 'Red', 'G': 'Green', 'X': 'Black'}
reverse_color_map = {v: k for k, v in color_map.items()}

class TapeTransformationGame:
    def __init__(self, tasks):
        self.tasks = tasks
        self.current_task_index = 0
        self.output_tape = []
        self.setup_ui()

    def setup_ui(self):
        self.task_label = widgets.Label()
        self.example_pairs = widgets.VBox()
        self.input_tape = widgets.HBox()
        self.output_tape_widget = widgets.HBox()
        
        self.color_buttons = [widgets.Button(description=color.capitalize(), layout=widgets.Layout(width='80px', height='40px')) for color in color_map.values()]
        for button in self.color_buttons:
            button.on_click(self.on_color_button_click)
        
        self.delete_button = widgets.Button(description="Delete", layout=widgets.Layout(width='80px', height='40px'))
        self.delete_button.on_click(self.on_delete_button_click)
        
        self.check_button = widgets.Button(description="Check")
        self.check_button.on_click(self.on_check_button_click)

        self.previous_button = widgets.Button(description="Previous")
        self.previous_button.on_click(self.on_previous_button_click)

        self.next_button = widgets.Button(description="Next")
        self.next_button.on_click(self.on_next_button_click)
        
        self.result_label = widgets.Label()
        
        self.button_panel = widgets.HBox(self.color_buttons + [self.delete_button])
        
        self.main_layout = widgets.VBox([
            self.task_label,
            self.example_pairs,
            widgets.Label("Input:", width='80px'),
            self.input_tape,
            widgets.Label("Output:", width='80px'),
            self.output_tape_widget,
            self.button_panel,
            widgets.HBox([self.check_button, self.previous_button, self.next_button]),
            self.result_label
        ])
        
        display(self.main_layout)
        self.update_display()

    def create_cell(self, color):
        button = widgets.Button(layout=widgets.Layout(width='40px', height='40px'))
        button.style.button_color = color_map.get(color)
        return button

    def update_display(self):
        task = self.tasks[self.current_task_index]
        task_name = list(task.keys())[0]
        task_data = task[task_name]
        
        self.task_label.value = f"Task {self.current_task_index + 1}"
        
        # Display first 4 pairs as examples
        example_pairs = []
        for i in range(4):
            input_tape = task_data[i][0]
            output_tape = task_data[i][1]
            input_cells = widgets.HBox([self.create_cell(color) for color in input_tape])
            output_cells = widgets.HBox([self.create_cell(color) for color in output_tape])
            example_pairs.append(widgets.VBox([
                widgets.Label(f"Example {i+1}:"),
                widgets.HBox([widgets.Label("Input:", width='80px'), input_cells]),
                widgets.HBox([widgets.Label("Output:", width='80px'), output_cells])
            ]))
        self.example_pairs.children = example_pairs
        
        # Display the 5th input tape
        input_tape = task_data[4][0]
        self.input_tape.children = [self.create_cell(color) for color in input_tape]
        
        # Display the user's output tape
        self.output_tape_widget.children = [self.create_cell(color) for color in self.output_tape]
        
        self.previous_button.disabled = True if self.current_task_index == 0 else False
        self.next_button.disabled = True if self.current_task_index == len(self.tasks) - 1 else False

    def on_color_button_click(self, button):
        color = reverse_color_map[button.description]
        self.output_tape.append(color)
        self.update_display()

    def on_delete_button_click(self, button):
        if self.output_tape:
            self.output_tape.pop()
            self.update_display()

    def on_check_button_click(self, button):
        task = self.tasks[self.current_task_index]
        task_data = list(task.values())[0]
        correct_output = task_data[4][1]  # The 5th pair's output
        
        if ''.join(self.output_tape) == correct_output:
            self.result_label.value = "Correct."
        else:
            self.result_label.value = "Incorrect, try again."

    def on_previous_button_click(self, button):
        self.current_task_index -= 1        
        self.output_tape = []
        self.result_label.value = ""
        self.update_display()
        
    def on_next_button_click(self, button):
        self.current_task_index += 1        
        self.output_tape = []
        self.result_label.value = ""
        self.update_display()

In [3]:
with open('tasks.txt', 'r') as file:
    data = file.read()
tasks_list = data.strip().split('\n\n')
tasks = []

for idx, task in enumerate(tasks_list):
    task_name = f'task_{idx}'
    task_data = []
    for line in task.split('\n'):
        groups = [group.strip() for group in line.split('-')]
        task_data.append(groups)
    tasks.append({task_name: task_data})

# Create and start the game
game = TapeTransformationGame(tasks)